# Analysis of an individual VMP profile

Import packages.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as io
import utils

Load data.

In [ ]:
vmp_profile = io.loadmat(
    "vmp_profile_SPAMEX_2014.mat", squeeze_me=True
)

Where is this profile located?

In [ ]:
proj = projection = ccrs.Orthographic(vmp_profile["lon"], vmp_profile["lat"])

fig, ax = plt.subplots(subplot_kw=dict(projection=proj))
ax.set_global()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.plot(vmp_profile["lon"], vmp_profile["lat"], "ro", transform=ccrs.PlateCarree())
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)

fig.savefig(
    "profile_location_python.png",
    dpi=180,
    bbox_inches="tight",
    pad_inches=0.1,
)

What does the data look like?

In [ ]:
fig, ax = plt.subplots()
ax.plot(vmp_profile["temperature"], vmp_profile["depth"])
ax.set_xlabel("Temperature [deg C]")
ax.set_ylabel("Depth [m]")
ax.invert_yaxis()

In [ ]:
fig, ax = plt.subplots()
ax.semilogx(vmp_profile["dissipation"], vmp_profile["depth"])
ax.set_xlabel(r"$\varepsilon$ [W kg-1]")
ax.set_ylabel("Depth [m]")
ax.invert_yaxis()

In [ ]:
fig, ax = plt.subplots()
ax.plot(vmp_profile["N_squared"], vmp_profile["depth"])
ax.set_xlabel("N2 [s-2]")
ax.set_ylabel("Depth [m]")
ax.invert_yaxis()

Buoynacy frequency has some spikes! We need to get rid of those.

In [ ]:
N2_clean = utils.despike(vmp_profile["N_squared"])

In [ ]:
fig, ax = plt.subplots()
ax.plot(N2_clean, vmp_profile["depth"])
ax.set_xlabel("N2 [s-2]")
ax.set_ylabel("Depth [m]")
ax.invert_yaxis()

This looks a lot better.

But there are negative values and it seems noisy so we should smooth.

In [ ]:
N2_smooth = utils.smooth(N2_clean, win=60)

fig, ax = plt.subplots()
ax.plot(N2_clean, vmp_profile["depth"])
ax.plot(N2_smooth, vmp_profile["depth"])
ax.set_xlabel("N2 [s-2]")
ax.set_ylabel("Depth [m]")
ax.invert_yaxis()
ax.set_xlim(-0.2e-5, 1e-5)

Even better, but there are still a few negative values we want to exclude.

In [ ]:
# Clean up negative values and small values of N2
N2_smooth[N2_smooth < 1e-7] = np.NaN

Calculate the turbulent diffusivity.

In [ ]:
Krho = utils.rdiff(vmp_profile["dissipation"], N2_smooth, 0.2)

Save the result.

In [ ]:
fig, ax = plt.subplots()
ax.semilogx(Krho, vmp_profile["depth"])
ax.set_xlabel(r"$K_{\rho}$ [m2 s-1]")
ax.set_ylabel("Depth [m]")
ax.invert_yaxis()

fig.savefig(
    "diffusivity_python.png", dpi=180, bbox_inches="tight", pad_inches=0.1
)